# Read data from sql

In [1]:
%matplotlib inline
from sqlalchemy import create_engine
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [2]:

db_credentials = json.loads(open('database_credentials.json').read())

In [3]:
cred_string = ('mysql+pymysql://' +
               db_credentials['username'] +
               ':' +
               db_credentials['user_pass'] +
               '@' +
               db_credentials['db_host'] +
               '/' +
               db_credentials['db_name']
              )

conn = create_engine(cred_string)
conn.table_names()
table_account, table_demographics, table_enquiries = conn.table_names()

In [28]:
table_contents = []
for table in conn.table_names():
    query = f"select * from {table}"
    table_content = pd.read_sql(query,conn)
    table_contents.append(table_content)

In [5]:
for table_content in table_contents:
    print(type(table_content))
    print(table_content.shape)
    

<class 'pandas.core.frame.DataFrame'>
(186329, 21)
<class 'pandas.core.frame.DataFrame'>
(23896, 83)
<class 'pandas.core.frame.DataFrame'>
(413188, 6)


In [6]:
for table_content in table_contents:
    
    print(table_content.head(5))

   dt_opened customer_no  upload_dt acct_type owner_indic  opened_dt  \
0  10-Nov-15       12265  20-Oct-15         6           1  09-Jun-13   
1  10-Nov-15       12265  20-Oct-15        10           1  25-May-12   
2  10-Nov-15       12265  20-Oct-15        10           1  22-Mar-12   
3  20-Jul-15       15606  09-Jul-15        10           1  13-Jan-06   
4  20-Jul-15       15606  09-Jul-15         6           1  18-Jan-15   

  last_paymt_dt  closed_dt reporting_dt high_credit_amt  ... amt_past_due  \
0     30-Jun-14  05-Jul-14    30-Sep-15           20900  ...                
1     06-Sep-15               03-Oct-15           16201  ...                
2     31-Aug-15               30-Sep-15           41028  ...                
3                26-Jul-07    31-Jan-09           93473  ...                
4     05-May-15               31-May-15           20250  ...                

                                     paymenthistory1  \
0  """STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXX

In [7]:
for table_name, table_content in zip(conn.table_names(), table_contents):
    print(table_name)
    print(table_content.columns)
    

Cust_Account
Index(['dt_opened', 'customer_no', 'upload_dt', 'acct_type', 'owner_indic',
       'opened_dt', 'last_paymt_dt', 'closed_dt', 'reporting_dt',
       'high_credit_amt', 'cur_balance_amt', 'amt_past_due', 'paymenthistory1',
       'paymenthistory2', 'paymt_str_dt', 'paymt_end_dt', 'creditlimit',
       'cashlimit', 'rateofinterest', 'paymentfrequency',
       'actualpaymentamount'],
      dtype='object')
Cust_Demographics
Index(['dt_opened', 'customer_no', 'entry_time', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12',
       'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22',
       'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27',
       'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32',
       'feature_33', 'feature_34'

In [8]:
table_contents[0].dtypes[['customer_no', 'dt_opened']]

customer_no    object
dt_opened      object
dtype: object

In [9]:
table_contents[1].dtypes[['customer_no', 'dt_opened']]

customer_no    object
dt_opened      object
dtype: object

In [10]:
set(table_contents[0].columns).intersection(table_contents[1].columns)

{'customer_no', 'dt_opened'}

In [11]:
data_joined = pd.concat([table_contents[0], table_contents[1], table_contents[2]], join='outer')

In [12]:
data_joined.shape

(623413, 105)

In [13]:
data_joined['Bad_label'].sort_values().unique()

array(['0', '1', nan], dtype=object)

In [14]:
data_joined['Bad_label'].isnull().sum()

599517

In [15]:
a = pd.DataFrame({
 'id': [0, 1, 2, 0 , 1],
    'a': ['a0', 'a1', 'a2', 'a0', 'a1']
})

b = pd.DataFrame({
    'id': [0, 1, 0 , 1],
    'b': ['b0', 'b1', 'b0', 'b1'],
    'label': [0, 1, 0, 1]
})

c = pd.DataFrame({
    'id': [0, 1],
    'c': ['c0', 'c1']
})

d = pd.concat([a, b,c ], join='outer')
for i in d.groupby('id'):
    print(i)

(0,      a    b    c  id  label
0   a0  NaN  NaN   0    NaN
3   a0  NaN  NaN   0    NaN
0  NaN   b0  NaN   0    0.0
2  NaN   b0  NaN   0    0.0
0  NaN  NaN   c0   0    NaN)
(1,      a    b    c  id  label
1   a1  NaN  NaN   1    NaN
4   a1  NaN  NaN   1    NaN
1  NaN   b1  NaN   1    1.0
3  NaN   b1  NaN   1    1.0
1  NaN  NaN   c1   1    NaN)
(2,     a    b    c  id  label
2  a2  NaN  NaN   2    NaN)


In [16]:
e = d.groupby('id').agg(list)
e

,a,b,c,label
id,,,,
0,"[a0, a0, nan, nan, nan]","[nan, nan, b0, b0, nan]","[nan, nan, nan, nan, c0]","[nan, nan, 0.0, 0.0, nan]"
1,"[a1, a1, nan, nan, nan]","[nan, nan, b1, b1, nan]","[nan, nan, nan, nan, c1]","[nan, nan, 1.0, 1.0, nan]"
2,[a2],[nan],[nan],[nan]


In [17]:
e

,a,b,c,label
id,,,,
0,"[a0, a0, nan, nan, nan]","[nan, nan, b0, b0, nan]","[nan, nan, nan, nan, c0]","[nan, nan, 0.0, 0.0, nan]"
1,"[a1, a1, nan, nan, nan]","[nan, nan, b1, b1, nan]","[nan, nan, nan, nan, c1]","[nan, nan, 1.0, 1.0, nan]"
2,[a2],[nan],[nan],[nan]


In [18]:
data = data_joined.groupby('customer_no').agg(list)

In [19]:
data.shape

(23896, 104)

In [20]:
x = pd.Series(data.iloc[6]['Bad_label'])
pd.to_numeric(x[x.notnull()]).max()
        

1

In [21]:
for column in data.columns:
    print(column)
    print(data[column].head(1).iloc[0])

Bad_label
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '0', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
acct_type
['10', '10', '10', '10', '10', '10', '2', '2', '5', '6', '2', '2', '10', '10', '5', '5', '6', '6', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
actualpaymentamount
['', '25100', '', '', '', '45986', '', '', '', '', '', '', '', '', '', '', '', '', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
amt_past_due
['2528846', '', '', '', '', '', '', '', '', '', '', '', '', '9363', '', '', '', '', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
cashlimit
['', '', '', '', '', '168000', '', '', '', '', '', '', '', '', '', '', '', '', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
closed_dt
['', '', '05-Mar

In [22]:
table_contents[0]['customer_no'].unique().shape

(23896,)

In [23]:
table_contents[1]['customer_no'].unique().shape

(23896,)

In [24]:
table_contents[2]['customer_no'].unique().shape

(23896,)

In [25]:
data.to_csv("data/data_joined.csv")

In [29]:
for table_name, table_content in zip(conn.table_names(), table_contents):
    print(table_name)
    table_content.to_csv(f"data/{table_name}.csv")

Cust_Account
Cust_Demographics
Cust_Enquiry
